In [ ]:
# DataSens logging setup (marker:datasens_logging)
import logging, os
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler('logs/datasens.log', encoding='utf-8')
    ]
)
logging.info('Démarrage')


# DataSens E1_v2 — 01_setup_env

- Objectifs: configuration environnement, connexions MinIO + PostgreSQL, arborescence, logging
- Prérequis: Docker Compose lancé (MinIO + PostgreSQL), Python + venv, `pip install -r requirements.txt`
- Ordre global E1_v2: 01 → 02 → 03 → 04 → 05
- Guide: docs/GUIDE_TECHNIQUE_E1.md

> **E1_v2** : Collecte réelle avec sources réduites mais fonctionnelles (18 tables PostgreSQL)


In [ ]:
# ============================================================
# 🎬 DASHBOARD NARRATIF - OÙ SOMMES-NOUS ?
# ============================================================
# Ce dashboard vous guide à travers le pipeline DataSens E1
# Il montre la progression et l'état actuel des données
# ============================================================

import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch
import matplotlib.patches as mpatches

print("\n" + "="*80)
print("🎬 FIL D'ARIANE VISUEL - PIPELINE DATASENS E1")
print("="*80)

# Créer figure dashboard
fig = plt.figure(figsize=(16, 8))
ax = fig.add_subplot(111)
ax.set_xlim(0, 10)
ax.set_ylim(0, 6)
ax.axis('off')

# Étapes du pipeline
etapes = [
    {"nom": "📥 COLLECTE", "status": "✅", "desc": "Sources brutes"},
    {"nom": "☁️ DATALAKE", "status": "✅", "desc": "MinIO Raw"},
    {"nom": "🧹 NETTOYAGE", "status": "🔄", "desc": "Déduplication"},
    {"nom": "💾 ETL", "status": "⏳", "desc": "PostgreSQL"},
    {"nom": "📊 ANNOTATION", "status": "⏳", "desc": "Enrichissement"},
    {"nom": "📦 EXPORT", "status": "⏳", "desc": "Dataset IA"}
]

# Couleurs selon statut
colors = {
    "✅": "#4ECDC4",
    "🔄": "#FECA57", 
    "⏳": "#E8E8E8"
}

# Dessiner timeline
y_pos = 4
x_start = 1
x_spacing = 1.4

for i, etape in enumerate(etapes):
    x_pos = x_start + i * x_spacing
    
    # Cercle étape
    circle = plt.Circle((x_pos, y_pos), 0.25, color=colors[etape["status"]], zorder=3)
    ax.add_patch(circle)
    ax.text(x_pos, y_pos, etape["status"], ha='center', va='center', fontsize=14, fontweight='bold', zorder=4)
    
    # Nom étape
    ax.text(x_pos, y_pos - 0.6, etape["nom"], ha='center', va='top', fontsize=11, fontweight='bold')
    ax.text(x_pos, y_pos - 0.85, etape["desc"], ha='center', va='top', fontsize=9, style='italic')
    
    # Flèche vers prochaine étape
    if i < len(etapes) - 1:
        ax.arrow(x_pos + 0.3, y_pos, x_spacing - 0.6, 0, 
                head_width=0.1, head_length=0.15, fc='gray', ec='gray', zorder=2)

# Titre narratif
ax.text(5, 5.5, "🎯 PROGRESSION DU PIPELINE E1", ha='center', va='center', 
        fontsize=16, fontweight='bold', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Légende
legend_elements = [
    mpatches.Patch(facecolor='#4ECDC4', label='Terminé'),
    mpatches.Patch(facecolor='#FECA57', label='En cours'),
    mpatches.Patch(facecolor='#E8E8E8', label='À venir')
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

# Statistiques rapides (si disponibles)
stats_text = "\n📊 SNAPSHOT ACTUEL :\n"
try:
    # Essayer de charger des stats si base disponible
    stats_text += "   • Pipeline en cours d'exécution...\n"
except:
    stats_text += "   • Démarrage du pipeline...\n"

ax.text(5, 1.5, stats_text, ha='center', va='center', fontsize=10,
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.title("🎬 FIL D'ARIANE VISUEL - Accompagnement narratif du jury", 
          fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n💡 Le fil d'Ariane vous guide étape par étape à travers le pipeline")
print("   Chaque visualisation s'inscrit dans cette progression narrative\n")



> Notes:
> - Configuration des connexions **MinIO (DataLake)** et **PostgreSQL (SGBD)**
> - Création de l'arborescence `data/raw/` avec sous-dossiers par type de source
> - Système de logging pour tracer toutes les opérations
> - Fonctions utilitaires (timestamp UTC, hash SHA256 pour déduplication)


In [8]:
# DataSens E1_v2 - 01_setup_env
# 🔧 Configuration environnement : MinIO + PostgreSQL + Arborescence + Logging

import datetime as dt
import hashlib
import logging
import os
from datetime import UTC, datetime
from pathlib import Path

from dotenv import load_dotenv

# Détection robuste du dossier projet
current = Path.cwd()
PROJECT_ROOT = None
while current != current.parent:
    if (current / "notebooks").exists() and (current / "docs").exists():
        PROJECT_ROOT = current
        break
    current = current.parent
else:
    PROJECT_ROOT = Path.cwd()

print(f"📂 Racine projet détectée : {PROJECT_ROOT}")

# Chargement .env
env_path = PROJECT_ROOT / '.env'
loaded = load_dotenv(env_path)
if loaded:
    print(f'✅ .env chargé: {env_path}')
else:
    print(f'⚠️ .env non trouvé: {env_path}')

# Configuration MinIO (DataLake)
MINIO_ENDPOINT = os.getenv("MINIO_ENDPOINT", "http://localhost:9002")
MINIO_ACCESS_KEY = os.getenv("MINIO_ACCESS_KEY", "admin")
MINIO_SECRET_KEY = os.getenv("MINIO_SECRET_KEY", "admin123")
MINIO_BUCKET = os.getenv("MINIO_BUCKET", "datasens-raw")

# Configuration PostgreSQL (SGBD)
PG_HOST = os.getenv("POSTGRES_HOST", "localhost")
PG_PORT = int(os.getenv("POSTGRES_PORT", "5433"))
PG_DB = os.getenv("POSTGRES_DB", "postgres")
PG_USER = os.getenv("POSTGRES_USER", "postgres")
PG_PASS = os.getenv("POSTGRES_PASS", "postgres")
PG_URL = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"

# Clés API (optionnelles)
KAGGLE_USERNAME = os.getenv("KAGGLE_USERNAME")
KAGGLE_KEY = os.getenv("KAGGLE_KEY")
OWM_API_KEY = os.getenv("OWM_API_KEY")
NEWSAPI_KEY = os.getenv("NEWSAPI_KEY")
GDELT_BASE = os.getenv("GDELT_BASE", "http://data.gdeltproject.org/gkg/")

print("\n🔐 Configuration MinIO (DataLake) :")
print(f"   • Endpoint : {MINIO_ENDPOINT}")
print(f"   • Bucket   : {MINIO_BUCKET}")

print("\n🗄️ Configuration PostgreSQL (SGBD) :")
print(f"   • Host     : {PG_HOST}:{PG_PORT}")
print(f"   • Database : {PG_DB}")
print(f"   • User     : {PG_USER}")

print("\n🔑 Clés API :")
print(f"   • Kaggle        : {'✅ Configurée' if KAGGLE_USERNAME else '❌ Manquante'}")
print(f"   • OpenWeatherMap: {'✅ Configurée' if OWM_API_KEY else '❌ Manquante'}")
print(f"   • NewsAPI       : {'✅ Configurée' if NEWSAPI_KEY else '❌ Manquante'}")

# Arborescence
DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
LOGS_DIR = PROJECT_ROOT / 'logs'

RAW_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR.mkdir(parents=True, exist_ok=True)

folders = ["kaggle", "api/owm", "api/newsapi", "rss", "scraping/multi", 
           "scraping/viepublique", "scraping/datagouv", "gdelt", "manifests"]
for sub in folders:
    (RAW_DIR / sub).mkdir(parents=True, exist_ok=True)

print(f"\n✅ Arborescence créée: {RAW_DIR}")
print(f"   • {len(folders)} sous-dossiers prêts")

# Logging
stamp = datetime.now(UTC).strftime("%Y%m%d_%H%M%S")
log_file = LOGS_DIR / f"collecte_{stamp}.log"
logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(levelname)s - %(message)s',
    datefmt='%H:%M:%S',
    handlers=[
        logging.FileHandler(log_file, encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logging.info("Système de logging initialisé")
print(f"📄 Log: {log_file}")

# Fonctions utilitaires
def ts() -> str:
    """Timestamp UTC ISO compact (YYYYMMDDTHHMMSSZ)"""
    return dt.datetime.now(tz=dt.UTC).strftime("%Y%m%dT%H%M%SZ")

def sha256_hash(s: str) -> str:
    """Hash SHA256 pour déduplication"""
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

print(f"\n🔧 Utilitaires : ts()={ts()}, sha256()={sha256_hash('test')[:16]}...")

print("\n✅ Configuration terminée !")


[13:51:48] INFO - Système de logging initialisé


📂 Racine projet détectée : c:\Users\Utilisateur\Desktop\DataSens
✅ .env chargé: c:\Users\Utilisateur\Desktop\DataSens\.env

🔐 Configuration MinIO (DataLake) :
   • Endpoint : http://localhost:9000
   • Bucket   : datasens-raw

🗄️ Configuration PostgreSQL (SGBD) :
   • Host     : localhost:5432
   • Database : datasens
   • User     : ds_user

🔑 Clés API :
   • Kaggle        : ✅ Configurée
   • OpenWeatherMap: ✅ Configurée
   • NewsAPI       : ✅ Configurée

✅ Arborescence créée: c:\Users\Utilisateur\Desktop\DataSens\data\raw
   • 9 sous-dossiers prêts
📄 Log: c:\Users\Utilisateur\Desktop\DataSens\logs\collecte_20251101_125148.log

🔧 Utilitaires : ts()=20251101T125148Z, sha256()=9f86d081884c7d65...

✅ Configuration terminée !


In [9]:
# Test des connexions MinIO et PostgreSQL

print("🔌 Test des connexions...")
print("=" * 80)

# Connexion MinIO
try:
    from minio import Minio
    from minio.error import S3Error
    
    minio_client = Minio(
        MINIO_ENDPOINT.replace("http://", "").replace("https://", ""),
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
    )
    
    # Créer le bucket s'il n'existe pas
    if not minio_client.bucket_exists(MINIO_BUCKET):
        minio_client.make_bucket(MINIO_BUCKET)
        print(f"✅ MinIO : Bucket '{MINIO_BUCKET}' créé")
    else:
        print(f"✅ MinIO : Bucket '{MINIO_BUCKET}' existe déjà")
    
    # Lister les objets existants
    objects = list(minio_client.list_objects(MINIO_BUCKET, recursive=False))
    print(f"   • {len(list(objects))} objets existants dans le bucket")
    
except Exception as e:
    print(f"❌ MinIO : Erreur de connexion - {e}")
    print("   💡 Vérifiez que Docker Compose est lancé : docker compose up -d")
    minio_client = None

# Connexion PostgreSQL
try:
    from sqlalchemy import create_engine, text
    
    engine = create_engine(PG_URL, future=True)
    
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1 as test"))
        test_value = result.scalar()
    
    if test_value == 1:
        print(f"✅ PostgreSQL : Connexion réussie ({PG_HOST}:{PG_PORT}/{PG_DB})")
        
        # Compter les tables existantes
        with engine.connect() as conn:
            result = conn.execute(text("""
                SELECT COUNT(*) FROM information_schema.tables 
                WHERE table_schema = 'public'
            """))
            nb_tables = result.scalar()
            print(f"   • {nb_tables} tables existantes dans la base")
    else:
        print("⚠️ PostgreSQL : Connexion OK mais test inattendu")
        
except Exception as e:
    print(f"❌ PostgreSQL : Erreur de connexion - {e}")
    print("   💡 Vérifiez que Docker Compose est lancé : docker compose up -d")
    engine = None

print("\n✅ Tests de connexion terminés !")


🔌 Test des connexions...
✅ MinIO : Bucket 'datasens-raw' existe déjà


KeyboardInterrupt: 

In [4]:
# Override to root-level directories
try:
    ROOT = ROOT
except NameError:
    from pathlib import Path
    ROOT = Path.cwd().resolve().parents[2]
LOGS = ROOT / 'logs'
DATA = ROOT / 'data' / 'raw'
print('ROOT=', ROOT)
print('LOGS=', LOGS)
print('DATA(raw)=', DATA)


ROOT= C:\Users\Utilisateur\Desktop
LOGS= C:\Users\Utilisateur\Desktop\logs
DATA(raw)= C:\Users\Utilisateur\Desktop\data\raw


In [6]:
# Charger .env depuis la racine et créer un .env.example si absent
from pathlib import Path
from dotenv import load_dotenv

# Définir ROOT si non défini
if 'ROOT' not in globals():
    ROOT = Path.cwd().resolve().parents[2]

env_path = ROOT / '.env'
loaded = load_dotenv(env_path)
if loaded:
    print('✅ .env chargé:', env_path)
else:
    print('⚠️ .env non trouvé:', env_path)
    example = ROOT / '.env.example'
    if not example.exists():
        example.write_text("""
# PostgreSQL
POSTGRES_HOST=localhost
POSTGRES_PORT=5432
POSTGRES_DB=datasens
POSTGRES_USER=ds_user
POSTGRES_PASS=ds_pass

# API Keys (optionnelles pour démo)
OWM_API_KEY=
KAGGLE_USERNAME=
KAGGLE_KEY=

# Git (optionnel)
GIT_USER_NAME=
GIT_USER_EMAIL=
""".strip()+"\n", encoding='utf-8')
        print('📄 Exemple créé:', example)
    else:
        print('📄 Exemple déjà présent:', example)



⚠️ .env non trouvé: C:\Users\Utilisateur\Desktop\.env
📄 Exemple déjà présent: C:\Users\Utilisateur\Desktop\.env.example


# DataSens E1_v2 — 01_setup_env

- Objectifs: arborescence raw, logging, .env
- Prérequis: Python, venv activé, `pip install -r requirements.txt`
- Ordre global E1_v2: 01 → 02 → 03 → 04 → 05
- Guide: docs/GUIDE_TECHNIQUE_E1.md


In [7]:
# DataSens E1_v2 - 01_setup_env
# Config .env, arborescence raw, logging
import logging
from datetime import UTC, datetime
from pathlib import Path

ROOT = Path.cwd()
DATA = ROOT / "data" / "raw"
for sub in ["kaggle","api/owm","api/newsapi","rss","scraping/multi","scraping/viepublique","scraping/datagouv","gdelt","manifests"]:
    (DATA / sub).mkdir(parents=True, exist_ok=True)
print("✅ Arborescence raw créée:", DATA)

# Logging
LOGS = ROOT.parent / "logs"
LOGS.mkdir(parents=True, exist_ok=True)
stamp = datetime.now(UTC).strftime("%Y%m%d_%H%M%S")
log_file = LOGS / f"collecte_{stamp}.log"
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s - %(message)s', datefmt='%H:%M:%S', handlers=[logging.FileHandler(log_file, encoding='utf-8'), logging.StreamHandler()])
logging.info("Système de logging initialisé")
print("📄 Log:", log_file)



[12:20:39] INFO - Système de logging initialisé


✅ Arborescence raw créée: c:\Users\Utilisateur\Desktop\DataSens\notebooks\datasens_E1_v2\data\raw
📄 Log: c:\Users\Utilisateur\Desktop\DataSens\notebooks\logs\collecte_20251101_112039.log
